# Агент "Куда сходить в Алматы"

Агент парсит sxodim.com и советует куда сходить в Алматы.

**Стек:**
- Crawl4AI - парсинг сайта
- OpenAI GPT-4o-mini - структурирование данных и агент
- LangChain - фреймворк для агента с tools calling
- Qwen2.5-1.5B + QLoRA - SFT для дружелюбного стиля (Apple M4 Pro MPS)

## 1. Импорты и настройка

In [5]:
import json
from typing import List
from datetime import datetime

from dotenv import load_dotenv
from openai import OpenAI

from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig

from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()

DATA_FILE = "sxodim_data.json"
print("Импорты загружены")

Импорты загружены


## 2. Парсинг sxodim.com с Crawl4AI

In [6]:
URLS_TO_PARSE = [
    "https://sxodim.com/almaty",
]

async def crawl_sxodim():
    """Парсим несколько страниц sxodim.com"""
    all_content = {}
    
    browser_config = BrowserConfig(headless=True)
    crawler_config = CrawlerRunConfig(
        word_count_threshold=10,
        remove_overlay_elements=True,
    )
    
    async with AsyncWebCrawler(config=browser_config) as crawler:
        for url in URLS_TO_PARSE:
            print(f"Парсинг: {url}")
            try:
                result = await crawler.arun(url=url, config=crawler_config)
                if result.success:
                    page_name = url.split("/")[-1] or "main"
                    all_content[page_name] = {
                        "url": url,
                        "markdown": result.markdown[:15000] if result.markdown else "",
                    }
                    print(f"  Получено {len(result.markdown or '')} символов")
                else:
                    print(f"  Ошибка: {result.error_message}")
            except Exception as e:
                print(f"  Исключение: {e}")
    
    return all_content

# Запуск парсинга
raw_data = await crawl_sxodim()
print(f"\nСпарсено {len(raw_data)} страниц")

[INIT].... → Crawl4AI 0.8.0 

Парсинг: https://sxodim.com/almaty


[FETCH]... ↓ https://sxodim.com/almaty                                                                            |
✓ | ⏱: 6.40s 

[SCRAPE].. ◆ https://sxodim.com/almaty                                                                            |
✓ | ⏱: 0.03s 

[COMPLETE] ● https://sxodim.com/almaty                                                                            |
✓ | ⏱: 6.44s 

  Получено 16897 символов

Спарсено 1 страниц


## 3. Структурирование данных с GPT-4o-mini

In [7]:
client = OpenAI()

def extract_places_from_markdown(markdown: str, category: str) -> List[dict]:
    """Извлекаем структурированные данные из markdown с помощью GPT-4o-mini"""
    
    prompt = f"""Извлеки информацию о местах/событиях из следующего текста.
Категория: {category}

Верни JSON массив с объектами, содержащими:
- name: название места/события
- description: краткое описание (1-2 предложения)
- category: категория (ресторан, бар, концерт, выставка, развлечение, детское, свидание и т.д.)
- address: адрес если есть
- price: цена/ценовая категория если указана
- date: дата события если указана
- tags: теги для поиска (массив строк)

Если информации нет - ставь null.
Извлеки максимум 15 мест.

Текст:
{markdown[:8000]}

Ответь ТОЛЬКО валидным JSON массивом, без комментариев."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            response_format={"type": "json_object"}
        )
        
        content = response.choices[0].message.content
        data = json.loads(content)
        
        # Если вернулся объект с ключом, извлекаем массив
        if isinstance(data, dict):
            for key in data:
                if isinstance(data[key], list):
                    return data[key]
            return []
        return data if isinstance(data, list) else []
        
    except Exception as e:
        print(f"Ошибка извлечения для {category}: {e}")
        return []

# Структурируем все данные
structured_data = {
    "places": [],
    "parsed_at": datetime.now().isoformat(),
    "source": "sxodim.com/almaty"
}

for page_name, page_data in raw_data.items():
    print(f"Обработка: {page_name}")
    places = extract_places_from_markdown(page_data["markdown"], page_name)
    for place in places:
        place["source_page"] = page_name
    structured_data["places"].extend(places)
    print(f"  Извлечено {len(places)} мест")

print(f"\nВсего мест: {len(structured_data['places'])}")

Обработка: almaty
  Извлечено 15 мест

Всего мест: 15


In [8]:
# Сохраняем в JSON файл
with open(DATA_FILE, "w", encoding="utf-8") as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=2)

print(f"Данные сохранены в {DATA_FILE}")
print("\nПример данных:")
for place in structured_data["places"][:3]:
    print(json.dumps(place, ensure_ascii=False, indent=2))

Данные сохранены в sxodim_data.json

Пример данных:
{
  "name": "Миша Зейферт: сольный стендап-концерт «Потомственный мужчина»",
  "description": "Сольный стендап-концерт Миши Зейферта.",
  "category": "концерт",
  "address": "ул. Каирбекова, 35А",
  "price": "от 5000 тенге",
  "date": "27 февраля в 19:00, 21:30",
  "tags": [
    "стендуп",
    "концерт"
  ],
  "source_page": "almaty"
}
{
  "name": "Концерт Maher Zain",
  "description": "Концерт известного исполнителя Maher Zain.",
  "category": "концерт",
  "address": "пр. Достык, 56",
  "price": "от 10 000 тенге",
  "date": "28 марта в 20:00",
  "tags": [
    "концерт",
    "музыка"
  ],
  "source_page": "almaty"
}
{
  "name": "Фестиваль Winter fest в Oi-Qaragai",
  "description": "Зимний фестиваль с различными активностями.",
  "category": "фестиваль",
  "address": "Талгарский район, Алматинская область",
  "price": "от 5000 тенге",
  "date": "31 января в 13:00",
  "tags": [
    "фестиваль",
    "зима"
  ],
  "source_page": "almaty"

## 4. Загрузка данных и создание Tools

In [9]:
# Загружаем данные из файла
with open(DATA_FILE, "r", encoding="utf-8") as f:
    places_data = json.load(f)

PLACES = places_data["places"]
print(f"Загружено {len(PLACES)} мест из {DATA_FILE}")

Загружено 15 мест из sxodim_data.json


In [10]:
@tool
def search_places(query: str) -> str:
    """Поиск мест по ключевым словам в названии, описании и тегах.
    
    Args:
        query: Поисковый запрос (например: 'концерт', 'ресторан', 'дети')
    """
    query_lower = query.lower()
    results = []
    
    for place in PLACES:
        searchable = " ".join([
            place.get("name", ""),
            place.get("description", ""),
            place.get("category", ""),
            " ".join(place.get("tags", []) or [])
        ]).lower()
        
        if query_lower in searchable:
            results.append(place)
    
    if not results:
        return f"Не найдено мест по запросу '{query}'"
    
    output = f"Найдено {len(results)} мест по запросу '{query}':\n\n"
    for p in results[:5]:
        output += f"- **{p.get('name', 'Без названия')}**\n"
        if p.get('description'):
            output += f"  {p['description']}\n"
        if p.get('address'):
            output += f"  Адрес: {p['address']}\n"
        if p.get('price'):
            output += f"  Цена: {p['price']}\n"
        output += "\n"
    
    return output


@tool  
def get_places_by_category(category: str) -> str:
    """Получить места по категории.
    
    Args:
        category: Категория места (ресторан, бар, концерт, выставка, развлечение, детское, свидание)
    """
    category_lower = category.lower()
    results = []
    
    for place in PLACES:
        place_cat = (place.get("category", "") or "").lower()
        place_tags = [t.lower() for t in (place.get("tags", []) or [])]
        
        if category_lower in place_cat or category_lower in place_tags:
            results.append(place)
    
    if not results:
        return f"Не найдено мест в категории '{category}'"
    
    output = f"Места в категории '{category}' ({len(results)} шт.):\n\n"
    for p in results[:5]:
        output += f"- **{p.get('name', 'Без названия')}**\n"
        if p.get('description'):
            output += f"  {p['description']}\n"
        output += "\n"
    
    return output


@tool
def get_events() -> str:
    """Получить список актуальных событий (концерты, выставки, мероприятия)."""
    events = []
    event_categories = ["концерт", "выставка", "мероприятие", "фестиваль", "спектакль", "шоу"]
    
    for place in PLACES:
        cat = (place.get("category", "") or "").lower()
        tags = [t.lower() for t in (place.get("tags", []) or [])]
        
        if any(e in cat for e in event_categories) or any(e in tags for e in event_categories):
            events.append(place)
    
    if not events:
        return "Актуальных событий не найдено"
    
    output = f"Актуальные события ({len(events)} шт.):\n\n"
    for e in events[:7]:
        output += f"- **{e.get('name', 'Без названия')}**\n"
        if e.get('description'):
            output += f"  {e['description']}\n"
        if e.get('date'):
            output += f"  Дата: {e['date']}\n"
        if e.get('price'):
            output += f"  Цена: {e['price']}\n"
        output += "\n"
    
    return output


@tool
def get_recommendations(occasion: str) -> str:
    """Получить рекомендации для определенного случая.
    
    Args:
        occasion: Повод (свидание, с детьми, выходные, вечеринка, романтика)
    """
    occasion_mapping = {
        "свидание": ["свидание", "романтика", "романтический", "уютный", "ресторан"],
        "дети": ["дети", "детский", "семейный", "развлечение", "парк"],
        "выходные": ["развлечение", "парк", "прогулка", "отдых", "кино"],
        "вечеринка": ["бар", "клуб", "ночной", "караоке", "вечеринка"],
        "романтика": ["романтика", "свидание", "ужин", "уютный"],
    }
    
    keywords = occasion_mapping.get(occasion.lower(), [occasion.lower()])
    results = []
    
    for place in PLACES:
        searchable = " ".join([
            place.get("category", "") or "",
            " ".join(place.get("tags", []) or []),
            place.get("description", "") or ""
        ]).lower()
        
        if any(kw in searchable for kw in keywords):
            results.append(place)
    
    if not results:
        return f"Не найдено рекомендаций для '{occasion}'"
    
    output = f"Рекомендации для '{occasion}' ({len(results)} мест):\n\n"
    for p in results[:5]:
        output += f"- **{p.get('name', 'Без названия')}**\n"
        if p.get('description'):
            output += f"  {p['description']}\n"
        if p.get('address'):
            output += f"  Адрес: {p['address']}\n"
        output += "\n"
    
    return output


@tool
def list_all_categories() -> str:
    """Показать все доступные категории мест."""
    categories = set()
    for place in PLACES:
        if place.get("category"):
            categories.add(place["category"])
    
    if not categories:
        return "Категории не найдены"
    
    return "Доступные категории:\n" + "\n".join(f"- {c}" for c in sorted(categories))


tools = [search_places, get_places_by_category, get_events, get_recommendations, list_all_categories]
print(f"Создано {len(tools)} инструментов")

Создано 5 инструментов


## 5. Создание агента с памятью

In [11]:
# Промпт с дружелюбным стилем
SYSTEM_PROMPT = """Ты дружелюбный гид по Алматы! Твоя задача - помогать людям находить интересные места и события в городе.

Правила:
1. Всегда используй инструменты для поиска актуальной информации
2. Отвечай тепло и с энтузиазмом, но не переборщи
3. Давай конкретные рекомендации с деталями
4. Если не нашел подходящих мест - предложи альтернативы
5. Помни контекст разговора и предыдущие запросы пользователя

Ты можешь искать места, события, рестораны и развлечения в Алматы."""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Создаем агента
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Агент создан")

Агент создан


In [12]:
class AlmatyGuideAgent:
    """Агент-гид по Алматы с памятью"""
    
    def __init__(self, executor: AgentExecutor):
        self.executor = executor
        self.chat_history: List = []
    
    def chat(self, user_input: str) -> str:
        """Отправить сообщение агенту"""
        result = self.executor.invoke({
            "input": user_input,
            "chat_history": self.chat_history
        })
        
        # Сохраняем в историю
        self.chat_history.append(HumanMessage(content=user_input))
        self.chat_history.append(AIMessage(content=result["output"]))
        
        return result["output"]
    
    def clear_history(self):
        """Очистить историю разговора"""
        self.chat_history = []
        print("История очищена")

guide = AlmatyGuideAgent(agent_executor)
print("Гид по Алматы готов к работе!")

Гид по Алматы готов к работе!


## 6. Цикл обработки запросов

In [13]:
def print_response(response: str):
    """Красивый вывод ответа"""
    print("\n" + "="*60)
    print("ОТВЕТ:")
    print("="*60)
    print(response)
    print("="*60 + "\n")

def interactive_chat():
    """Интерактивный чат с агентом"""
    print("\n" + "*"*60)
    print("  ГАЙД ПО АЛМАТЫ - Куда сходить?")
    print("*"*60)
    print("Команды: 'выход' - завершить, 'очистить' - очистить историю")
    print("*"*60 + "\n")
    
    while True:
        try:
            user_input = input("Вы: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() in ["выход", "exit", "quit", "q"]:
                print("\nДо свидания! Хорошего отдыха в Алматы!")
                break
            
            if user_input.lower() in ["очистить", "clear"]:
                guide.clear_history()
                continue
            
            response = guide.chat(user_input)
            print_response(response)
            
        except KeyboardInterrupt:
            print("\n\nДо свидания!")
            break

# Раскомментируйте для интерактивного режима:
# interactive_chat()

## 7. Тестирование агента с вопросами

In [14]:
# Тестовые вопросы
TEST_QUESTIONS = [
    "Куда сходить на выходных?",
    "Что нового открылось в Алматы?",
    "Посоветуй место для свидания",
    "Куда сводить ребенка?",
    "Какие концерты будут?",
    "Где вкусно поесть?"
]

# Собираем ответы агента
agent_responses = []
guide.clear_history()

for question in TEST_QUESTIONS:
    print(f"\n{'='*60}")
    print(f"ВОПРОС: {question}")
    print('='*60)
    
    response = guide.chat(question)
    agent_responses.append({
        "question": question,
        "answer": response
    })
    
    print(f"\nОТВЕТ:\n{response}")
    print('='*60)
    guide.clear_history()  # Очищаем для независимых ответов

print(f"\n\nВсего собрано {len(agent_responses)} ответов")

История очищена

ВОПРОС: Куда сходить на выходных?


> Entering new AgentExecutor chain...

Invoking: `get_events` with `{}`


Актуальные события (12 шт.):

- **Миша Зейферт: сольный стендап-концерт «Потомственный мужчина»**
  Сольный стендап-концерт Миши Зейферта.
  Дата: 27 февраля в 19:00, 21:30
  Цена: от 5000 тенге

- **Концерт Maher Zain**
  Концерт известного исполнителя Maher Zain.
  Дата: 28 марта в 20:00
  Цена: от 10 000 тенге

- **Фестиваль Winter fest в Oi-Qaragai**
  Зимний фестиваль с различными активностями.
  Дата: 31 января в 13:00
  Цена: от 5000 тенге

- **Концерт Icegergert в Алматы**
  Концерт группы Icegergert.
  Дата: 3 апреля в 19:00
  Цена: 18 000 тенге

- **Спектакль «Рок-н-ролл на закате» в Алматы**
  Спектакль о рок-н-ролле.
  Дата: 30 января в 19:00
  Цена: от 19 000 тенге

- **Батыр аманаты және камералық оркестр: үлкен трибьют-концерт**
  Трибьют-концерт с участием камерного оркестра.
  Дата: 30 апреля в 19:00
  Цена: 12 500 теңгеден начиная

- **Cольный

## 8. Сохранение в agent_examples.md

In [15]:
# Сохраняем в markdown файл
EXAMPLES_FILE = "agent_examples.md"

markdown_content = "# Примеры диалогов с агентом \"Куда сходить в Алматы\"\n\n"
markdown_content += f"*Дата генерации: {datetime.now().strftime('%Y-%m-%d %H:%M')}*\n\n"
markdown_content += "---\n\n"

for i, qa in enumerate(agent_responses, 1):
    markdown_content += f"## Пример {i}\n\n"
    markdown_content += f"**Вопрос:** {qa['question']}\n\n"
    markdown_content += f"**Ответ:**\n\n{qa['answer']}\n\n"
    markdown_content += "---\n\n"

markdown_content += "## Использованные инструменты агента\n\n"
markdown_content += "| Tool | Описание |\n"
markdown_content += "|------|----------|\n"
markdown_content += "| `search_places` | Поиск по ключевым словам |\n"
markdown_content += "| `get_places_by_category` | Фильтр по категории |\n"
markdown_content += "| `get_events` | Актуальные события |\n"
markdown_content += "| `get_recommendations` | Рекомендации по поводу |\n"
markdown_content += "| `list_all_categories` | Список категорий |\n\n"
markdown_content += "---\n\n"
markdown_content += "*Данные получены с sxodim.com/almaty*\n"

with open(EXAMPLES_FILE, "w", encoding="utf-8") as f:
    f.write(markdown_content)

print(f"Примеры сохранены в {EXAMPLES_FILE}")
print(f"\nПервые 500 символов файла:\n{'-'*40}")
print(markdown_content[:500])

Примеры сохранены в agent_examples.md

Первые 500 символов файла:
----------------------------------------
# Примеры диалогов с агентом "Куда сходить в Алматы"

*Дата генерации: 2026-01-29 02:31*

---

## Пример 1

**Вопрос:** Куда сходить на выходных?

**Ответ:**

На выходных в Алматы вас ждут несколько интересных событий! Вот что я нашел:

1. **Сольный стендап-концерт Миши Зейферта "Потомственный мужчина"**
   - **Дата:** 27 февраля
   - **Время:** 19:00 и 21:30
   - **Цена:** от 5000 тенге
   - Это отличная возможность посмеяться и провести вечер в компании юмора!

2. **Фестиваль Winter fest в Oi-


## 9. Создание SFT датасета с GPT-4o-mini

Генерируем дружелюбные ответы на основе данных из sxodim_data.json

In [16]:
# Загружаем данные для генерации SFT датасета
with open(DATA_FILE, "r", encoding="utf-8") as f:
    places_data = json.load(f)

places = places_data["places"]
print(f"Загружено {len(places)} мест для генерации SFT датасета")

# Дружелюбный системный промпт для GPT-4o-mini
FRIENDLY_SYSTEM_PROMPT = """Ты дружелюбный и энергичный гид по Алматы! 

Твой стиль общения:
- Говори тепло, с энтузиазмом и позитивом
- Используй эмодзи в меру (1-2 на ответ)
- Давай конкретные рекомендации с деталями
- Добавляй личные "советы от местного"
- Заканчивай вопросом или предложением помощи

Пример хорошего ответа:
"Отличный выбор! 🎭 Концерт Therr Maitz пройдет 17 марта в 20:00. Билеты от 25000 тенге. 
Советую прийти пораньше - там отличный бар в фойе! Хотите узнать, где поужинать рядом?"
"""

def generate_friendly_response(question: str, context: str) -> str:
    """Генерируем дружелюбный ответ через GPT-4o-mini"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": FRIENDLY_SYSTEM_PROMPT},
                {"role": "user", "content": f"Контекст о местах в Алматы:\n{context}\n\nВопрос: {question}"}
            ],
            temperature=0.8,
            max_tokens=300
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Ошибка генерации: {e}")
        return None

# Подготавливаем контекст из данных
places_context = "\n".join([
    f"- {p['name']}: {p.get('description', '')} | Категория: {p.get('category', '')} | "
    f"Цена: {p.get('price', 'не указана')} | Дата: {p.get('date', 'не указана')}"
    for p in places[:10]
])

print("Контекст для генерации:")
print(places_context[:500] + "...")

Загружено 15 мест для генерации SFT датасета
Контекст для генерации:
- Миша Зейферт: сольный стендап-концерт «Потомственный мужчина»: Сольный стендап-концерт Миши Зейферта. | Категория: концерт | Цена: от 5000 тенге | Дата: 27 февраля в 19:00, 21:30
- Концерт Maher Zain: Концерт известного исполнителя Maher Zain. | Категория: концерт | Цена: от 10 000 тенге | Дата: 28 марта в 20:00
- Фестиваль Winter fest в Oi-Qaragai: Зимний фестиваль с различными активностями. | Категория: фестиваль | Цена: от 5000 тенге | Дата: 31 января в 13:00
- Концерт Icegergert в Алматы: ...


In [17]:
# Вопросы для SFT датасета
SFT_QUESTIONS = [
    "Куда сходить на выходных?",
    "Что нового открылось в Алматы?",
    "Посоветуй место для свидания",
    "Куда сводить ребенка?",
    "Какие концерты будут?",
    "Где вкусно поесть?",
    "Что посмотреть в Алматы?",
    "Куда сходить вечером?",
    "Где провести время с друзьями?",
    "Какие фестивали будут?",
    "Посоветуй культурную программу",
    "Где можно послушать живую музыку?",
    "Куда сходить на стендап?",
    "Какие спектакли идут?",
    "Что интересного в этом месяце?"
]

# Генерируем SFT датасет
sft_dataset = []

print("Генерация SFT датасета...")
print("="*50)

for i, question in enumerate(SFT_QUESTIONS, 1):
    print(f"[{i}/{len(SFT_QUESTIONS)}] {question}")
    
    answer = generate_friendly_response(question, places_context)
    
    if answer:
        sft_dataset.append({
            "instruction": question,
            "input": "",
            "output": answer
        })
        print(f"  ✓ Ответ сгенерирован ({len(answer)} символов)")
    else:
        print(f"  ✗ Ошибка генерации")

print("="*50)
print(f"Создано {len(sft_dataset)} примеров для SFT")

Генерация SFT датасета...
[1/15] Куда сходить на выходных?
  ✓ Ответ сгенерирован (814 символов)
[2/15] Что нового открылось в Алматы?
  ✓ Ответ сгенерирован (593 символов)
[3/15] Посоветуй место для свидания
  ✓ Ответ сгенерирован (710 символов)
[4/15] Куда сводить ребенка?
  ✓ Ответ сгенерирован (787 символов)
[5/15] Какие концерты будут?
  ✓ Ответ сгенерирован (833 символов)
[6/15] Где вкусно поесть?
  ✓ Ответ сгенерирован (978 символов)
[7/15] Что посмотреть в Алматы?
  ✓ Ответ сгенерирован (1082 символов)
[8/15] Куда сходить вечером?
  ✓ Ответ сгенерирован (722 символов)
[9/15] Где провести время с друзьями?
  ✓ Ответ сгенерирован (737 символов)
[10/15] Какие фестивали будут?
  ✓ Ответ сгенерирован (390 символов)
[11/15] Посоветуй культурную программу
  ✓ Ответ сгенерирован (846 символов)
[12/15] Где можно послушать живую музыку?
  ✓ Ответ сгенерирован (944 символов)
[13/15] Куда сходить на стендап?
  ✓ Ответ сгенерирован (386 символов)
[14/15] Какие спектакли идут?
  ✓ Ответ сген

In [18]:
# Сохраняем SFT датасет
SFT_FILE = "orpo_friendly_dataset.json"

with open(SFT_FILE, "w", encoding="utf-8") as f:
    json.dump(sft_dataset, f, ensure_ascii=False, indent=2)

print(f"SFT датасет сохранен в {SFT_FILE}")
print(f"\nПример из датасета:")
print(json.dumps(sft_dataset[0], ensure_ascii=False, indent=2))

SFT датасет сохранен в orpo_friendly_dataset.json

Пример из датасета:
{
  "instruction": "Куда сходить на выходных?",
  "input": "",
  "output": "Ух ты, сколько интересных событий в Алматы на выходных! 🎉 Вот несколько классных вариантов:\n\n1. **Сольный стендап-концерт Миши Зейферта** - 27 февраля, в 19:00 и 21:30. Билеты от 5000 тенге. Если вы любите хорошую комедию, это отличный выбор! Не забудьте прийти пораньше и насладиться атмосферой.\n\n2. **Спектакль «Состояние Танго»** - 5 февраля в 20:00. Это музыкальный спектакль о танго, который обещает быть очень эмоциональным и драйвовым! Билеты от 19 000 тенге.\n\n3. **Фестиваль Winter fest в Oi-Qaragai** - 31 января в 13:00. Если хотите провести день на свежем воздухе, это мероприятие предлагает множество активностей и развлечений. Билеты от 5000 тенге.\n\nЛично я бы посоветовал стендап, это отличное средство снять напряжение и просто хорошо повеселиться! 😊 Какой вариант вам ближе? Могу помочь с деталями!"
}


## 10. SFT обучение Qwen2.5-1.5B + QLoRA

Обучаем модель на дружелюбном датасете с использованием GPU (Apple M4 Pro MPS)

In [27]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# Проверяем доступность MPS (Apple Silicon GPU)
if torch.backends.mps.is_available():
    device = "mps"
    print("✓ Apple Silicon GPU (MPS) доступен!")
else:
    device = "cpu"
    print("⚠ MPS недоступен, используем CPU")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

✓ Apple Silicon GPU (MPS) доступен!
PyTorch version: 2.10.0
Device: mps


In [28]:
# Загружаем SFT датасет
with open(SFT_FILE, "r", encoding="utf-8") as f:
    sft_data = json.load(f)

# Форматируем для SFT
def format_for_sft(example):
    return {
        "text": f"### Вопрос:\n{example['instruction']}\n\n### Ответ:\n{example['output']}"
    }

formatted_data = [format_for_sft(ex) for ex in sft_data]
train_dataset = Dataset.from_list(formatted_data)

print(f"Датасет для обучения: {len(train_dataset)} примеров")
print(f"\nПример форматированных данных:")
print(train_dataset[0]["text"][:300] + "...")

Датасет для обучения: 15 примеров

Пример форматированных данных:
### Вопрос:
Куда сходить на выходных?

### Ответ:
Ух ты, сколько интересных событий в Алматы на выходных! 🎉 Вот несколько классных вариантов:

1. **Сольный стендап-концерт Миши Зейферта** - 27 февраля, в 19:00 и 21:30. Билеты от 5000 тенге. Если вы любите хорошую комедию, это отличный выбор! Не забу...


In [29]:
# Загружаем модель Qwen2.5-1.5B с оптимизацией для MPS
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

print(f"Загрузка модели: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Используем bfloat16 для ускорения на MPS
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,  # Быстрее чем float32
    device_map={"": device},
)

print(f"✓ Модель загружена: {model.num_parameters():,} параметров")
print(f"✓ Dtype: {next(model.parameters()).dtype}")
print(f"✓ Device: {next(model.parameters()).device}")

Загрузка модели: Qwen/Qwen2.5-1.5B-Instruct
✓ Модель загружена: 1,543,714,304 параметров
✓ Dtype: torch.bfloat16
✓ Device: mps:0


In [30]:
# QLoRA конфигурация - оптимизированная
lora_config = LoraConfig(
    r=8,  # Уменьшили с 16 - быстрее, качество почти то же
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],  # Только attention - быстрее
)

# Применяем LoRA к модели
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Оптимизированные аргументы для MPS
training_args = TrainingArguments(
    output_dir="./sft_almaty_guide",
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Увеличили - лучше загрузка GPU
    gradient_accumulation_steps=1,  # Убрали накопление - быстрее
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=2,  # Фиксированное число вместо ratio
    logging_steps=1,
    save_strategy="epoch",
    bf16=True,  # Включаем bfloat16 для тренировки
    optim="adamw_torch",
    report_to="none",
    dataloader_num_workers=0,  # Для MPS лучше 0
    dataloader_pin_memory=False,  # Не нужно для MPS
)

print("✓ Оптимизированная конфигурация для MPS готова")

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705
✓ Оптимизированная конфигурация для MPS готова


In [31]:
# Запуск SFT обучения с мониторингом
import time

print("="*50)
print("Запуск SFT обучения на Apple MPS (bfloat16)")
print("="*50)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)

# Замеряем время
start_time = time.time()

# Обучение
trainer.train()

training_time = time.time() - start_time

# Сохраняем модель
OUTPUT_DIR = "./sft_almaty_guide_final"
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("="*50)
print(f"✓ SFT обучение завершено!")
print(f"✓ Время обучения: {training_time:.1f} секунд ({training_time/60:.1f} мин)")
print(f"✓ Модель сохранена в {OUTPUT_DIR}")
print("="*50)

Запуск SFT обучения на Apple MPS (bfloat16)


Adding EOS to train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Step,Training Loss
1,1.944200
2,2.030100
3,2.009200
4,2.080900
5,1.900300
6,2.015800
7,2.027700
8,2.002300
9,2.033000
10,1.986900


✓ SFT обучение завершено!
✓ Время обучения: 165.3 секунд (2.8 мин)
✓ Модель сохранена в ./sft_almaty_guide_final


## 11. Тестирование обученной модели

In [32]:
from peft import PeftModel

# Загружаем базовую модель с bfloat16
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map={"": device},
)

# Загружаем обученные LoRA веса
trained_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
trained_model.eval()

print(f"✓ Обученная модель загружена с {OUTPUT_DIR}")

def generate_local_response(question: str) -> str:
    """Генерация ответа от обученной локальной модели"""
    prompt = f"### Вопрос:\n{question}\n\n### Ответ:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = trained_model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Ответ:")[-1].strip()

✓ Обученная модель загружена с ./sft_almaty_guide_final


In [34]:
# Тестовые вопросы для обученной модели
LOCAL_TEST_QUESTIONS = [
    "Куда сходить на выходных в Алмате?",
    "Какие концерты будут в Алмате?",
    "Посоветуй место для свидания в Алмате",
]

print("="*60)
print("ТЕСТИРОВАНИЕ ОБУЧЕННОЙ МОДЕЛИ (Qwen2.5-1.5B + QLoRA)")
print("="*60)

for question in LOCAL_TEST_QUESTIONS:
    print(f"\n{'─'*60}")
    print(f"ВОПРОС: {question}")
    print(f"{'─'*60}")
    
    response = generate_local_response(question)
    print(f"ОТВЕТ:\n{response}")

print(f"\n{'='*60}")
print("✓ Тестирование завершено!")
print("="*60)

ТЕСТИРОВАНИЕ ОБУЧЕННОЙ МОДЕЛИ (Qwen2.5-1.5B + QLoRA)

────────────────────────────────────────────────────────────
ВОПРОС: Куда сходить на выходных в Алмате?
────────────────────────────────────────────────────────────
ОТВЕТ:
На выходные можно сходить на Казахский фестиваль моды и дизайна, который проходит в рамках Казахского международного кинофестиваля. Также стоит посетить выставки искусственных рукавиц и ботинок. Кроме того, можно отправиться на охоту или лошадь-пастушью эстафету. В парке имени Жакарата есть множество мероприятий для всех возрастов. Музей архитектуры и истории города также предлагает интересный набор событий. 

Все эти места являются отличными местами для отдыха и развлечений во время выходных в Алмате.

### Примечание:

Несмотря на то что информация была актуальной на момент написания вопроса, некоторые события могут измениться из-за пандемии коронавируса. Исп

────────────────────────────────────────────────────────────
ВОПРОС: Какие концерты будут в Алмате?
────

## Итог

**Реализовано:**
1. Парсинг sxodim.com через Crawl4AI
2. Структурирование данных в JSON через GPT-4o-mini
3. Агент с 5 инструментами (tools calling) на LangChain
4. Память разговора (chat history)
5. Тестирование агента с 6 вопросами
6. Сохранение примеров в agent_examples.md
7. Генерация SFT датасета с дружелюбными ответами (GPT-4o-mini)
8. SFT обучение Qwen2.5-1.5B + QLoRA на Apple M4 Pro (MPS)
9. Примеры работы обученной модели

**Файлы:**
- `sxodim_data.json` - спарсенные данные
- `agent_examples.md` - примеры диалогов агента
- `orpo_friendly_dataset.json` - SFT датасет
- `sft_almaty_guide_final/` - обученная модель